https://yangtavares.com/2017/07/31/creating-a-simple-overlay-for-pynq-z1-board-from-vivado-hlx/

In [1]:
#This notebook also uses the `(some) LaTeX environments for Jupyter`
#https://github.com/ProfFan/latex_envs wich is part of the
#jupyter_contrib_nbextensions package

from myhdl import *
from myhdlpeek import Peeker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sympy import *
init_printing()

import itertools

from IPython.display import clear_output


#https://github.com/jrjohansson/version_information
%load_ext version_information
%version_information myhdl, myhdlpeek, numpy, pandas, matplotlib, sympy, itertools, IPython

Software,Version
Python,3.6.2 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython,6.2.1
OS,Linux 4.15.0 30 generic x86_64 with debian stretch sid
myhdl,0.10
myhdlpeek,0.0.6
numpy,1.13.3
pandas,0.23.3
matplotlib,2.1.0
sympy,1.1.2.dev
itertools,The 'itertools' distribution was not found and is required by the application


In [2]:
#helper  functions to read in the .v and .vhd generated files into python
def VerilogTextReader(loc, printresult=True):
    with open(f'{loc}.v', 'r') as vText:
        VerilogText=vText.read()
    if printresult:
        print(f'***Verilog modual from {loc}.v***\n\n', VerilogText)
    return VerilogText

def VHDLTextReader(loc, printresult=True):
    with open(f'{loc}.vhd', 'r') as vText:
        VerilogText=vText.read()
    if printresult:
        print(f'***VHDL modual from {loc}.vhd***\n\n', VerilogText)
    return VerilogText

def ConstraintXDCTextReader(loc, printresult=True):
    with open(f'{loc}.xdc', 'r') as xdcText:
        ConstraintText=xdcText.read()
    if printresult:
        print(f'***Constraint file from {loc}.xdc***\n\n', ConstraintText)
    return ConstraintText

# IP  ClockDivider

## Non SoC Test

In [3]:
@block
def ClockDivider(Divisor, clkOut, clk,rst):
    """
    Simple Clock Divider based on the Digilint Clock Divider
    https://learn.digilentinc.com/Documents/262
    
    Input:
        Divisor(32 bit): the clock frequncy divide by value
        clk(bool): The input clock
        rst(bool): clockDivider Reset
    
    Ouput:
        clkOut(bool): the divided clock ouput
        count(32bit): the value of the internal counter
    """
    
    count_i=Signal(modbv(0)[32:])
    @always(clk.posedge, rst.posedge)
    def counter():
        if rst:
            count_i.next=0
        elif count_i==(Divisor-1):
            count_i.next=0
        else:
            count_i.next=count_i+1
    
    clkOut_i=Signal(bool(0))
    @always(clk.posedge, rst.posedge)
    def clockTick():
        if rst:
            clkOut_i.next=0
        elif count_i==(Divisor-1):
            clkOut_i.next=not clkOut_i
        else:
            clkOut_i.next=clkOut_i
        
        
    
    @always_comb
    def OuputBuffer():
        clkOut.next=clkOut_i
       
    return instances()
            
        

In [4]:
RefClkFreq=125e6
TargetClkFreq=40
DivsionFactor=int(RefClkFreq/TargetClkFreq)
DivsionFactor

In [5]:
Peeker.clear()
clk=Signal(bool(0)); Peeker(clk, 'clk')
Divisor=Signal(intbv(DivsionFactor)[32:]); Peeker(Divisor, 'Divisor')
clkOut=Signal(bool(0)); Peeker(clkOut, 'clkOut')
rst=Signal(bool(0)); Peeker(rst, 'rst')

DUT=ClockDivider(Divisor, clkOut, clk,rst)
DUT.convert()
VerilogTextReader('ClockDivider');

***Verilog modual from ClockDivider.v***

 // File: ClockDivider.v
// Generated by MyHDL 0.10
// Date: Tue Sep 11 23:32:46 2018


`timescale 1ns/10ps

module ClockDivider (
    Divisor,
    clkOut,
    clk,
    rst
);
// Simple Clock Divider based on the Digilint Clock Divider
// https://learn.digilentinc.com/Documents/262
// 
// Input:
//     Divisor(32 bit): the clock frequncy divide by value
//     clk(bool): The input clock
//     rst(bool): clockDivider Reset
// 
// Ouput:
//     clkOut(bool): the divided clock ouput
//     count(32bit): the value of the internal counter

input [31:0] Divisor;
output clkOut;
wire clkOut;
input clk;
input rst;

reg [31:0] count_i;
reg clkOut_i;



always @(posedge clk, posedge rst) begin: CLOCKDIVIDER_COUNTER
    if (rst) begin
        count_i <= 0;
    end
    else if (($signed({1'b0, count_i}) == ($signed({1'b0, Divisor}) - 1))) begin
        count_i <= 0;
    end
    else begin
        count_i <= (count_i + 1);
    end
end


always @(posedge clk

In [6]:
ConstraintXDCTextReader('ClockDivider');

***Constraint file from ClockDivider.xdc***

 ## PYNQ-Z1 Board Constraints for countLED.v
## Based on https://github.com/Xilinx/PYNQ/blob/master/sdbuild/boot_configs/Pynq-Z1-defconfig/constraints.xdc

## Clock signal 125 MHz

set_property -dict { PACKAGE_PIN H16   IOSTANDARD LVCMOS33 } [get_ports { clk }]; #IO_L13P_T2_MRCC_35 Sch=clk
create_clock -add -name sys_clk_pin -period 10.00 -waveform {0 5} [get_ports { clk }];



##LEDs
set_property -dict {PACKAGE_PIN R14 IOSTANDARD LVCMOS33} [get_ports {clkOut}]

## Buttons
set_property -dict { PACKAGE_PIN L19   IOSTANDARD LVCMOS33 } [get_ports { rst }]; ##btn[3]





## AXI SoC

The IP Project is `myClockDividerIP_v1_0`
After adding the Verilog Clock Divider Module under sources there are two addintal modules that where created with the IP that are the AXI Lite Slave IP Connection Header `myClockDividerIP_v1_0.v` and the AXI Slave BUS controler `myClockDividerIP_v1_0_S00_AXI_inst.v`